# IG360 Campaign - X by X
*Purpose:* Find a list of new accounts for target user to target for engagement
<br><br>
*Methodology*
<ul>
    <li>Select the most prominent X likers in target user's last X posts</li>
    <li>For each promiment liker, count the number of likes in their last X posts</li>
    <li>Add up likes for all users across their most recent posts - return the largest count</li>
</ul>

<br>
<br>

In [ ]:
# import IG360 data
from IG360.ig360_scrape import *

# import other key libraries
import pandas as pd
import numpy as np
import re
from pathlib import Path
from datetime import datetime


# set output folder
txt_dir_output = '_output/'
dir_output = Path(txt_dir_output)

# locate configuration file
config_file_path = 'config.ini'
config = init_config(config_file_path)

# set user and password (turned off by default; using config.ini)
#user_login = 'xx'
#user_password = 'xx'

# set target account and capture parameters
tgt_user = "worldofbenches"
xFactor = 5
max_recs = 20

In [ ]:
# Initialize scraper and login
#scraper = IG360Scrape(config, user_login, user_password)
scraper = IG360Scrape(config)
scraper.login()

Get list of accounts currently followed by target account

In [ ]:
scraper.scrape_following(tgt_user)
tgt_following_list = scraper.record_following_list

Select the most prominent X likers in target user's last X posts

In [ ]:
# get list of most recent X posts
scraper.scrape_post_list(tgt_user, xFactor)
tgt_posts = [i.split('/')[-2] for i in scraper.get_posts()]

# get details for each post
post_data = []
for tgt_post in tgt_posts:
    scraper.scrape_post(tgt_post)
    post_data.append(scraper.get_post_data())

In [ ]:
# create data sets for posts and likes
df_posts = pd.DataFrame()
df_post_likes = pd.DataFrame()
for postdat in post_data:
    # convert date time field
    postdat = list(postdat)
    postdat[1] = datetime.strptime(postdat[1], '%Y-%m-%dT%H:%M:%S.%fZ')
    
    # basic post information
    addon = pd.DataFrame([postdat[:6]], columns=['picture_id', 'post_date', 'poster', 'location', 'likes', 'post'])
    df_posts = df_posts.append(addon, ignore_index=True)
    
    # like matrix
    for lk in postdat[6]:
        if len(lk) > 0:
            addonl = pd.DataFrame([[postdat[0], postdat[1], lk]], columns=['picture_id', 'post_date', 'account'])
            df_post_likes = df_post_likes.append(addonl, ignore_index=True) 
            
# aggregate likers
top_likers = (df_post_likes.groupby(['account'])
                   .agg({'picture_id':'count', 'post_date':'max'})
                   .sort_values(['picture_id'], ascending=[False])
                   .reset_index(drop=False))
top_likers.columns = ['account','num_likes', 'last_post']

# add calcuation for days since last post
max_post = df_post_likes['post_date'].max()
top_likers['days_snc_lst'] = ((max_post - top_likers['last_post']) / np.timedelta64(1, 'D')).astype("int")

# capture top likers
lst_top_likers = list(top_likers['account'].head(xFactor))

# display top likers
top_likers[['account','num_likes', 'days_snc_lst']].head(xFactor)

For each promiment liker, count the number of likes in their last X posts

In [ ]:
# get list of most recent X posts for each of the top likers
l_posts = []
for ltop in lst_top_likers:
    scraper.scrape_post_list(ltop, xFactor)
    addon_l_posts = [i.split('/')[-2] for i in scraper.get_posts()]
    l_posts = l_posts + addon_l_posts
    
# get details for each liker post
l_post_data = []
for l_tgt_post in l_posts:
    scraper.scrape_post(l_tgt_post)
    l_post_data.append(scraper.get_post_data())

In [ ]:
l_df_posts = pd.DataFrame()
l_df_post_likes = pd.DataFrame()
for l_postdat in l_post_data:
    # convert date time field
    l_postdat = list(l_postdat)
    l_postdat[1] = datetime.strptime(l_postdat[1], '%Y-%m-%dT%H:%M:%S.%fZ')
    
    # basic post information
    addon = pd.DataFrame([l_postdat[:6]], columns=['picture_id', 'post_date', 'poster', 'location', 'likes', 'post'])
    l_df_posts = l_df_posts.append(addon, ignore_index=True)
    
    # like matrix
    for lk in l_postdat[6]:
        if len(lk) > 0:
            addonl = pd.DataFrame([[l_postdat[0], l_postdat[1], lk]], columns=['picture_id', 'post_date', 'account'])
            l_df_post_likes = l_df_post_likes.append(addonl, ignore_index=True) 
            
# aggregate likers
l_top_likers = (l_df_post_likes.groupby(['account'])
                   .agg({'picture_id':'count', 'post_date':'max'})
                   .sort_values(['picture_id'], ascending=[False])
                   .reset_index(drop=False))
l_top_likers.columns = ['account','total_likes', 'last_like']

# add calcuation for days since last post
max_post = l_df_post_likes['post_date'].max()
l_top_likers['days_snc_last_like'] = ((max_post - l_top_likers['last_like']) / np.timedelta64(1, 'D')).astype("int")

# remove target user from like list
l_top_likers = l_top_likers[l_top_likers['account']!=tgt_user]

# capture top prospects for following
rept_lst_top_likers = (l_top_likers[~l_top_likers['account'].isin(tgt_following_list)]
                       .head(max_recs)
                      )
rept_lst_top_likers